In [1]:
import pandas as pd
import numpy as np

In [5]:
from os import listdir
from os.path import isfile, join

path = "../price data/candles/"
files = [f for f in listdir(path) if isfile(join(path, f))]

In [6]:
files

['candles_eth-usd_24h.csv',
 'candles_atom-usd_24h.csv',
 'candles_xtz-usd_24h.csv',
 'candles_eos-usd_24h.csv',
 'candles_trx-usd_24h.csv',
 'candles_xrp-usd_24h.csv',
 'candles_ada-usd_24h.csv',
 'candles_link-usd_24h.csv',
 'candles_ltc-usd_24h.csv',
 'candles_xlm-usd_24h.csv',
 'candles_xmr-usd_24h.csv',
 'candles_neo-usd_24h.csv',
 'candles_etc-usd_24h.csv',
 'candles_btc-usd_24h.csv']

In [7]:

price_col = 'close'

#feature_names = [price_col,'sma10','sma21']
feature_names = [price_col]

for file in files:
    data = pd.DataFrame()

    #read data
    file_path = path+file
    d = pd.read_csv(file_path)
    d['timestamp'] = [pd.Timestamp(t) for t in d['timestamp']]
    
    #daily average
    d['day'] = [ts.strftime('%Y-%m-%d') for ts in d['timestamp'].to_list()]
    d = d.groupby(by=['day']).mean()
    d.index = [pd.Timestamp(ts) for ts in d.index.to_list()]
    d['day'] = d.index
    d['market'] = file[8:15]
    
    x = d[['market','day',price_col]].copy()
    
    #generate sma of price
    #x[feature_names[1]] = x.rolling(window=int(feature_names[1][-2:]))['close'].mean()
    #x[feature_names[2]] = x.rolling(window=int(feature_names[2][-2:]))['close'].mean()
    
    for feature in feature_names:
        x['log_'+feature] = np.log(x[feature])
        x['diff_'+feature] = x[feature].diff()
        x['log_diff_'+feature] = x['log_'+feature].diff()
        x['diff_ratio_'+feature] = (x['diff_'+feature]/x[feature])*100
    
    data = pd.concat([data,x],ignore_index=True)
    data.to_csv("generated_price_data/"+file[8:15]+"_daily.csv",index=False)